In [1]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
import base64
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
_ = load_dotenv() 

In [3]:
antropic_api_key = os.getenv("ANTHROPIC_API_KEY")

In [4]:
llm = ChatAnthropic(
    model_name='claude-sonnet-4-5-20250929',api_key=antropic_api_key)

llm

ChatAnthropic(model='claude-sonnet-4-5-20250929', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [5]:
llm_openai = ChatOpenAI(
            model='gpt-4o-mini',
            api_key=os.getenv("OPENAI_API_KEY")
        )

llm_openai.invoke('Hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-Cl83acU5ywGD0udNt0lW8xW5zKQBh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--40531bea-0bf0-4a2d-ad3f-7517b0749d10-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

image_path = Path("../data/test_images/IMG_2320.jpeg")  # Update with your image path
base64_image = encode_image(str(image_path))

In [12]:
prompt = """
Analyze the image of a store shelf containing various products, such as personal care items, cosmetics, or household goods. Your task is to identify and count each distinct product type based on the following criteria, treating each unique combination as a separate product type:

- Brand: Extract the exact brand name from visible labels or logos.
- Variant: Identify the variant or type based on label text, descriptions, or implied purpose (e.g., specific formulations, scents, or functions).
- Color Description: Note the primary colors of the packaging, liquid, cap, or design elements (e.g., "Blue bottle with white cap", "Red liquid in clear container").
- Size: Estimate the size or volume based on visible proportions or labeled values (e.g., "Small ~100ml", "Large ~200ml", "Medium box", or "Unknown" if not discernible).
- Packaging Type: Determine the packaging format (e.g., "Bottle", "Box", "Jar", "Tube", "Can").

Do not group products together even if they share the same brand; count each unique combination of brand, variant, color description, size, and packaging type separately. Use all available visual cues, including label text, imagery, and design patterns, to differentiate products. If labels are unreadable or unclear, describe the product based on visible features (e.g., "Unlabeled green bottle with floral design") and treat it as a distinct type if it doesn't match others.

Ignore non-product items such as shelves, price tags, dividers, or background elements. Estimate counts for partially obscured, stacked, or overlapping items by analyzing visible portions, arrangement, and patterns (e.g., extrapolate hidden items in uniform rows). If uncertainty exists, base estimates on the most likely number given the context.

Return the results strictly in JSON format, with no additional text outside the JSON. Use this exact structure:

{
  "products": [
    {
      "brand": "Exact brand name from label or 'Unknown' if not discernible",
      "variant": "Variant name or description based on label or implied type (e.g., 'Moisturizing Cream', 'Herbal Oil') or 'Unknown' if not discernible",
      "color_description": "Description of primary colors (e.g., 'Yellow bottle with silver cap', 'Green liquid in transparent container')",
      "size": "Approximate size or volume (e.g., 'Small ~100ml', 'Large ~200ml', 'Medium box', or 'Unknown' if not discernible)",
      "packaging_type": "Type of packaging (e.g., 'Bottle', 'Box', 'Jar', 'Tube', 'Can')",
      "count": Number of items of this exact type
    },
    ...
  ],
  "total_distinct_types": Total number of unique product types identified,
  "total_items": Sum of all product counts,
  "notes": "Brief observations or uncertainties (e.g., 'Some items partially obscured in back row; counts estimated', 'Labels unreadable for certain products')"
}

Ensure the JSON is valid, comprehensive, and lists products in a logical order (e.g., from top-left to bottom-right of the shelf) for clarity. Prioritize accuracy by cross-referencing all visible details, and omit categories or products if no relevant data is present.
"""

In [13]:
from langchain_core.messages import HumanMessage
message = HumanMessage(
    content=[
        {"type": "text", "text": prompt},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
    ]
)

In [15]:
response = llm_openai.invoke([message])
result = response.content
print(result)

```json
{
  "products": [
    {
      "brand": "Amla",
      "variant": "Amla Plus",
      "color_description": "Green bottle",
      "size": "Large ~200ml",
      "packaging_type": "Bottle",
      "count": 1
    },
    {
      "brand": "Amla",
      "variant": "Amla",
      "color_description": "Black bottle",
      "size": "Large ~200ml",
      "packaging_type": "Bottle",
      "count": 2
    },
    {
      "brand": "Herbal",
      "variant": "Jasmine",
      "color_description": "Transparent bottle with blue cap",
      "size": "Medium ~150ml",
      "packaging_type": "Bottle",
      "count": 1
    },
    {
      "brand": "7 Oils",
      "variant": "Hair Oil",
      "color_description": "Clear bottle with orange liquid and golden cap",
      "size": "Medium ~100ml",
      "packaging_type": "Bottle",
      "count": 1
    },
    {
      "brand": "Emami",
      "variant": "Kesh King Ayurvedic Oil",
      "color_description": "Green and black box with image",
      "size": "Medium box",

In [17]:
from langchain_core.messages import HumanMessage 

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_product_info(image_path):
    prompt = """
    Analyze the product image and extract key identifying features for product matching and comparison.

    Extract the following features:

    - Brand: Extract the exact brand name from visible labels or logos.
    - Product Name: Full product name or title as shown on the packaging.
    - Variant: Identify the variant or type (e.g., "Moisturizing", "Anti-dandruff", "Lavender scent").
    - Primary Colors: List the dominant colors of the packaging (e.g., ["Blue", "White", "Silver"]).
    - Packaging Type: Type of container (e.g., "Bottle", "Tube", "Jar", "Box", "Can").
    - Shape: Describe the container shape (e.g., "Cylindrical", "Rectangular", "Oval").
    - Size/Volume: Extract exact size/volume from label (e.g., "200ml", "500g") or estimate if not visible.
    - Logo/Brand Mark: Brief description of the brand logo appearance and position.
    - Key Text: Extract main visible text or claims on the label.
    - Distinctive Visual Elements: Any unique design features, patterns, or color combinations that make this product recognizable.

    If certain features are unclear or not visible, mark them as "Unknown".

    Return the results strictly in JSON format with this structure:

    {
    "product": {
        "brand": "Exact brand name or 'Unknown'",
        "product_name": "Full product name or 'Unknown'",
        "variant": "Variant/type or 'Unknown'",
        "primary_colors": ["List of dominant colors"],
        "packaging_type": "Container type",
        "shape": "Shape description",
        "size_volume": "Exact size/volume or estimated",
        "logo_description": "Brief logo description",
        "key_text": ["List of main visible text"],
        "distinctive_elements": ["Unique visual identifiers"]
    }
    }

    Ensure the JSON is valid. Focus on features that are most useful for matching this product against other product images.
    """
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
        
    message = HumanMessage(
    content=[
        {"type": "text", "text": prompt},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
        ]
    )
    
    response = llm_openai.invoke([message])
    result = response.content
    
    return result    

In [23]:
result = get_product_info('../data/test_images/cropped_image_IMG_2320_9.jpg')

In [24]:
print(result)

```json
{
    "product": {
        "brand": "Janet",
        "product_name": "Hair Fall Control Dark Henna Hair Oil",
        "variant": "Hair Fall Control",
        "primary_colors": ["Red", "White", "Green", "Pink"],
        "packaging_type": "Box",
        "shape": "Rectangular",
        "size_volume": "Unknown",
        "logo_description": "The brand logo features the name 'Janet' in a stylized font, positioned at the top.",
        "key_text": ["HAIR FALL CONTROL", "DARK HENNA HAIR OIL", "PARABEN FREE"],
        "distinctive_elements": ["Collage of plant images", "Bold text", "Paraben free label"]
    }
}
```


In [10]:
response = llm.invoke([message])
result = response.content
print(result)

```json
{
  "products": [
    {
      "brand": "Emami",
      "variant": "7 Oils in One - Hair Fall Control - Nourishing Aloe Vera+",
      "color_description": "Green transparent bottle with gold cap, white and gold label with aloe vera imagery",
      "size": "Unknown",
      "packaging_type": "Bottle",
      "count": 1
    }
  ],
  "total_distinct_types": 1,
  "total_items": 1,
  "notes": "Single product visible in image. The product is Emami 7 Oils in One hair oil with hair fall control formula containing Aloe Vera and other ingredients. Paraben-free formulation. Size not clearly visible on label."
}
```


In [9]:
import os 
from dotenv import load_dotenv

_ = load_dotenv()

gimini_api_key = os.getenv('GIMINI_API_KEY')

In [11]:
import requests
import base64
import json


def gemini_inference(image, prompt, model_id, temperature, api_key):
    """
    Perform Gemini inference using Google Gemini API with text + image input.
    """
    API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{model_id}:generateContent?key={api_key}"
    
    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt},
                    {"inlineData": {"mimeType": "image/jpeg", "data": image}},
                ],
            }
        ],
        "generationConfig": {"temperature": temperature}
    }

    headers = {"Content-Type": "application/json"}
    
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    
    result = response.json()
    
    return result["candidates"][0]["content"]["parts"][0]["text"]

if __name__ == "__main__":
    # Encode image to base64 in one line
    base64_image = base64.b64encode(open("image.png", "rb").read()).decode("utf-8")

    # Run inference
    output = gemini_inference(
        base64_image,
        prompt="Count the number of objects in the image.",
        model_id="gemini-2.5-flash", # Gemini variant
        temperature=0.0,
        api_key=gimini_api_key
    )

    print(output)

I found 5 objects in the image.


In [16]:
import requests
import base64
import json


def gemini_inference(image, prompt, model_id, temperature, api_key, function_declarations):
    """
    Perform Gemini inference using Google Gemini API with text + image input.
    Uses function calling with a provided schema.
    """
    API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{model_id}:generateContent?key={api_key}"

    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt},
                    {"inlineData": {"mimeType": "image/jpeg", "data": image}},
                ],
            }
        ],
        "generationConfig": {"temperature": temperature},
        "tools": [{"functionDeclarations": function_declarations}],
    }

    headers = {"Content-Type": "application/json"}
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    response.raise_for_status()

    result = response.json()
    fn_call = result["candidates"][0]["content"]["parts"][0]["functionCall"]

    return fn_call["args"]


if __name__ == "__main__":
    # Encode image to base64
    base64_image = base64.b64encode(open("../data/test_images/IMG_2329.jpeg", "rb").read()).decode("utf-8")

    # Define function schema with descriptions
    function_declarations = [
        {
            "name": "list_objects",
            "description": "Identify objects present in the given image.",
            "parameters": {
                "type": "object",
                "properties": {
                    "objects": {
                        "type": "array",
                        "description": "A list of objects detected in the image.",
                        "items": {
                            "type": "object",
                            "properties": {
                                "label": {
                                    "type": "string",
                                    "description": "The name of the detected object."
                                }
                            },
                            "required": ["label"],
                            "description": "Details about a single object detected in the image."
                        }
                    }
                },
                "required": ["objects"],
                "description": "The structured output containing all detected objects."
            }
        }
    ]

    # Run inference
    output = gemini_inference(
        base64_image,
        prompt="Identify the surface product in shelf in the image.",
        model_id="gemini-2.5-flash",
        temperature=0.0,
        api_key=gimini_api_key,
        function_declarations=function_declarations,
    )

    print(json.dumps(output, indent=2))

{
  "objects": [
    {
      "label": "ROMIK Olive Oil"
    },
    {
      "label": "Kumarka Hair Oil"
    },
    {
      "label": "Emami 7 Oils in One Hair Oil"
    },
    {
      "label": "Parachute Advansed Karapincha"
    },
    {
      "label": "Revlon Color N Care"
    },
    {
      "label": "Abha Herbal Black Henna Cr\u00e8me"
    },
    {
      "label": "Dreamron Color Cream"
    }
  ]
}
